<center>
    <h1><b>Multi-logistic Regression</b></h1>
    <h3><b>Softmax</b></h3>
</center>

In [1]:
#Load libraries
import random
import numpy as np
import pandas as pd
from sklearn import datasets
from tqdm import tqdm

---

In [2]:
#Load datas
df = pd.read_csv('./data/petal dataset/IRIS.csv')

#View top 5 row of data
df.head() 

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [11]:
#Data Preprocessing 
data = df[['petal_length', 'petal_width', 'species']]
data.head()

,petal_length,petal_width,species
0,1.4,0.2,Iris-setosa
1,1.4,0.2,Iris-setosa
2,1.3,0.2,Iris-setosa
3,1.5,0.2,Iris-setosa
4,1.4,0.2,Iris-setosa


In [12]:
#Encoding ordinal categorical feature
unique_species = data['species'].unique()
encoding_species = {}
for i in range(len(unique_species)):
    encoding_species[unique_species[i]] = i

#replace value of categorical
data['species'] = data['species'].replace(encoding_species)

C:\Users\admin\AppData\Local\Temp\ipykernel_12360\1613559914.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['species'] = data['species'].replace(encoding_species)


In [16]:
#Convert data to numpy
data = data.to_numpy()
data[:5]

array([[1.4, 0.2, 0. ],
       [1.4, 0.2, 0. ],
       [1.3, 0.2, 0. ],
       [1.5, 0.2, 0. ],
       [1.4, 0.2, 0. ]])

In [21]:
#Get num of samples
N = len(data)
N

150

In [24]:
#Shuffle data
np.random.shuffle(data)

#Get data train and test
train = data[:-30]
test = data[-30:]

#See data train and test
print('Train:\n', train[:5])
print('Test:\n', test[:5])

Train:
 [[6.7 2.  2. ]
 [3.  1.1 1. ]
 [5.7 2.3 2. ]
 [4.  1.  1. ]
 [4.2 1.5 1. ]]
Test:
 [[1.6 0.2 0. ]
 [4.3 1.3 1. ]
 [1.4 0.2 0. ]
 [5.6 2.4 2. ]
 [5.6 1.8 2. ]]


In [41]:
#Get features and targets train
X_train = train[:,:-1]
y_train = train[:,-1]

In [40]:
#Get features and targets test
X_test = test[:,:-1]
y_test = test[:,-1]

In [67]:
#Function convert categorical from number to one-hot encoder
def one_hot_encoder(y):
    y = y.astype(np.int8)
    n_cate = np.unique(y)
    encode = np.zeros((len(y), len(n_cate)))
    for i in range(len(encode)):
        encode[i, y[i]] = 1
    return encode